# A solution for NSUCRYPTO 2020
- Problem 8: _Collisions_
- By: _ndh_

**_Answer for Q1:_**

Let $|$ denotes block concatenation. Let $x$ be a block such that $f(x)$ is known. Let $m$ be a 2-block message $x|f(x)$. Then, $H(m) = f(f(x) \oplus x \oplus f(x)) \oplus f(x) = f(x) \oplus f(x) = 0$. Therefore, we have a simple algorithm for finding collisions of $H$ as follows:

-   Input: None.
-   Output: $m_1$ and $m_2$ such that $H(m_1) = H(m_2)$.


1.   Choose two random different blocks $b_1$, $b_2$ such that $f(b_1)$ and $f(b_2)$ are known (from the set of $10n$ obtained blocks).
2.   Output $b_1|f(b_1)$ and $b_2|f(b_2)$.

**_Answer for Q2:_**

Let $u_1 = m_1$,  $u_2 = m_1 \oplus m_2$, $u_3 = m_2 \oplus m_3$, ..., $u_k = m_{k-1} \oplus m_k$. We call $(u_1, u_2, ..., u_k)$ the u-form of $m$. Observe that $m_1 = u_1$, $m_2 = m_1 \oplus u_2 = u_1 \oplus u_2$, $m_3 = m_2 \oplus u_3 = u_1 \oplus u_2 \oplus u_3$, ..., $m_k = u_1 \oplus u_2 \;\oplus\;...\;\oplus\;u_k$, the hash function $H$ with respect to the u-form of $m$ is as follows (Figure 8.1):

_Figure 8.1_  


Here, two algorithms for finding the second preimage $m'$, given $m$ and $H(m)$, are proposed. The first one only uses $H(m)$, as it can find the preimage for any hash value. The second only uses $m$. Both of the algorithms exploit the XOR sum of $u_1, u_2, ..., u_k$ at the end of the hash function. 

From now on, blocks are treated as vectors in $\mathbb{F}_2^n$ and $b_i$ denotes a block such that $f(b_i)$ is known.

_Algorithm 1_:

-   Input: $H(m)$.  
-   Output: $m'$ such that $H(m') = H(m)$.


1.  Repeatedly pick a random vector $b_i$ until we have $n$ linearly independent vectors of the form $b_i + f(b_i)$. Those vectors form a basis $B$ for $\mathbb{F}_2^n$.
2.  Pick another vector $b_{-1}$. This vector will be used as input to the last call to $f$ (the right-most $f$, as in Figure 8.1).
3.  Solve the equation $Bx = H(m) + b_{-1} + f(b_{-1})$, which results in a subset $S$ of $B$ such that $\sum_S{(b_i+f(b_i))} = H(m) + b_{-1} + f(b_{-1})$. Name the vectors in $S$ as $b_1 + f(b_1), b_2 + f(b_2), ..., b_{|S|} + f(b_{|S|})$.
4.  If $S = \emptyset$ (this only happens when $H(m) = b_{-1} + f(b_{-1})$), return $b_{-1}$. Otherwise, let $(b_1, f(b_1) + b_2, f(b_2) + b_3, ..., f(b_{|S|-1}) + b_{|S|}$, $f(b_{|S|}) + b_{-1})$ be the u-form of $m'$, return $m'$.

The proof of correctness of Algorithm 1 just requires some simple arithmetic (see Figure 8.2). Note that in step 1, since we have $10n$ different $b_i$ to choose from, it's likely that $10n$ vectors of the form $(b_i + f(b_i))$ span $\mathbb{F}_2^n$ (see [this](https://math.stackexchange.com/questions/171966/whats-the-probability-a-subset-of-an-mathbb-f-2-vector-space-is-a-spanning-se) for more information) and we're able to construct a basis for $\mathbb{F}_2^n$ before the set of $b_i$ is exhausted.

_Figure 8.2_  


_Algorithm 2_:

-   Input: $m$.  
-   Output: $m' \ne m$ such that $H(m') = H(m)$.


1.  Repeatedly pick a random vector $b_i$ until we have a non-trivial linear relation between vectors of the form $b_i + f(b_i)$. Name the involved vectors $b_i$ as $b_1, b_2, ..., b_l$ ($l \ge 1$), we have $b_1 + f(b_1) + b_2 + f(b_2)\; + \;...\; + \;b_l + f(b_l) = 0$.
2.  Convert the input message to its u-form, obtain $(u_1, u_2, ..., u_k)$.
3.  Let the sequence $(b_1, f(b_1) + b_2, ..., f(b_l) + u_1, u_2, ..., u_k)$ be the u-form of $m'$. Return $m'$.

The proof of correctness of Algorithm 2 also just requires some simple arithmetic (see Figure 8.3). Note that in step 1, since we have $10n$ different vectors to choose from, and $10n > n$, such a linear relation always exists.

_Figure 8.3_  


**_Answer for Q3:_**

This question in fact asks us to implement the algorithms above. The implementations will be given in Python 3. They rely on [SageMath](https://www.sagemath.org/) for dealing with Linear Algebra stuffs.

Firstly, let's define some utilities:

In [1]:
from sage.all import GF, vector
from bitarray import bitarray
from typing import List

n = 256
F2 = GF(2)

def block_to_vec(block: bytes) -> vector:
    """Convert a block of n//8 bytes to a vector in F2^n."""
    assert len(block) == n // 8
    ba = bitarray()
    ba.frombytes(block)
    return vector(F2, ba)


def vec_to_block(v: vector) -> bytes:
    """Convert a vector in F2^n to a block of n//8 bytes."""
    assert len(v) == 256
    return bitarray(v).tobytes()


def bytes_to_uform(message: bytes) -> List[vector]:
    """Get the uform of a message."""
    assert len(message) * 8 % n == 0
    m = [block_to_vec(message[i:i + n // 8])
         for i in range(0, len(message), n // 8)]
    u = [None] * len(m)
    u[0] = m[0]
    for i in range(1, len(m)):
        u[i] = m[i] + m[i - 1]
    return u


def uform_to_bytes(u: List[vector]) -> bytes:
    """Construct a message from its uform."""
    m = [None] * len(u)
    m[0] = u[0]
    for i in range(1, len(m)):
        m[i] = u[i] + m[i - 1]
    return b''.join(vec_to_block(v) for v in m)

Now, collect the $(b_i, f(b_i))$ pairs. Since $b_i$ and $f(b_i)$ are given as integers, they need a little bit processing.

In [2]:
from sage.all import column_matrix

with open("Collisions-Values_of_F.txt") as r:
    lines = r.readlines()[1:]  # exclude the comment on the first line

b = []
fb = []
B_cols = []
for line in lines:
    bi, fbi = [block_to_vec(int(s).to_bytes(n // 8, "big"))
               for s in line.split(',')]
    b.append(bi)
    fb.append(fbi)
    B_cols.append(bi + fbi)

B = column_matrix(B_cols)

Here's the implementation of Algorithm 1:

In [3]:
def find_preimage_v1(target: bytes) -> bytes:
    """Return a message such that its hash value equals to `target`."""
    # step 1
    # In fact, we only need `B` to span F2^n. There's nothing much to do here.
    assert B.rank() == n

    # step 2
    # b[-1] is chosen, nothing to do here

    # step 3
    # Note that `S` is not explicitly needed.
    x = B.solve_right(block_to_vec(target) + b[-1] + fb[-1])
    # S = [b[i] + fb[i] for i in range(len(x)) if x[i] == 1]
    involved_b = [b[i] for i in range(len(x)) if x[i] == 1]
    involved_fb = [fb[i] for i in range(len(x)) if x[i] == 1]
    len_S = len(involved_b)

    # step 4
    if len_S == 0:
        return vec_to_block(b[-1])
    u = [None] * (len_S + 1)
    u[0] = involved_b[0]
    for i in range(1, len_S):
        u[i] = involved_fb[i - 1] + involved_b[i]
    u[-1] = involved_fb[-1] + b[-1]

    return uform_to_bytes(u)

Here's the implementation of Algorithm 2:

In [4]:
def find_preimage_v2(m: bytes) -> bytes:
    """Return _m such that _m != m and H(_m) == H(m)."""
    # step 1
    # just pick a non-zero vector in the right kernel of `B` to get a linear
    # relation
    x = B.right_kernel_matrix()[0]
    involved_b = [b[i] for i in range(len(x)) if x[i] == 1]
    involved_fb = [fb[i] for i in range(len(x)) if x[i] == 1]
    l = len(involved_b)

    # step 2
    u = bytes_to_uform(m)
    k = len(u)

    # step 3
    final_u = [None] * (k + l)
    final_u[0] = involved_b[0]
    for i in range(1, l):
        final_u[i] = involved_fb[i - 1] + involved_b[i]
    final_u[l] = involved_fb[-1] + u[0]
    for i in range(1, k):
        final_u[l + i] = u[i]

    return uform_to_bytes(final_u)

Here are two preimages found for the target message:

In [5]:
m = b"A random matrix is likely decent"
Hm_int = 96927213786383450510742783183865994351939043923041992908825186024229118761948
Hm = Hm_int.to_bytes(n // 8, "big")
print("m':", "h" + find_preimage_v1(Hm).hex())
print("m'':", "h" + find_preimage_v2(m).hex())

m': h0000000000000000000000000000000000000000000000000000000000000000ff1282609f458d732888e2736fd1b98cc36f809b1c116e77015b8d7d4d8996af6746a560ab3623e9b7b50e02f9bd772fd9f82ba5f7a15d4e089b6f88b43104ee22ccc22667624ce11b734e2f37e6705e4605259bc031fd1d81563be35f5cd8868a5e9c53faebc111dc1b90e395286ac249ca3ea214752b6490ea5f570c0b13fd2d72e89d545f0d26e08f3fa8d4ee5253b6ef37133629be312d3cc308a7f515ef3028d011b1f19556ce89f3ffa3955d80abdb655b8b07d3b22f5de8e360ac934392313683217c1a19ea0d7b42b5d6b1a20e0454e17f3a6928ee44c8a35ebdcf772f590d3fc11b961d7fca4c16220b18cfa88c8ea9f899c8d467828bcee6bbdcf2708355669fd5937342b2efc9121dc738fd7c7e47ee28bd8294419279f68b5fa312845bc98fab03660c3c145d1b3bf9d8ee3281559248501619de4c9e733b10251cbee834453111f2a172fb50fc03cb28308f1fe5a21087bd30bdc6fb679d76268c1d0c151051bbf8abb1b6bc003af527590819e3a67a8a04955af095017b6dff1d9fb53acfca71255816f7acc33cf1b27ee1ec73919739d23cddac409257cac12e40a4aa4b0bc2230476b31db0c65f540d34cc374e6d5db39703871af498e5ea127b7db506839a5ce5b44b8b67ced728e0b

When checked with the given oracle for $H$; $m$, $m'$ and $m''$ do produce the same output as expected.